<h1>Multiclass Text Classification </h1>
Pada project kali ini kita akan melakukan klasifikasi teks multikelas menggunakan LSTM.

Pada cell pertama impor library pandas dan ubah dataset menjadi dataframe. Kemudian buang kolom 'judul_film' karena kita hanya akan menggunakan sinopsis sebagai atribut untuk dilatih pada model.

In [1]:
import pandas as pd
df = pd.read_csv('imdb_indonesian_movies_2.csv')
df = df.drop(columns=['judul_film'])

In [2]:
df.head()
df

,ringkasan_sinopsis,genre
0,Raden Mas Said putra sulung Tumenggung Wilarik...,Drama
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,Drama
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,Drama
3,POL menceritakan kisah hidup yang luar biasa d...,Drama
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,Drama
...,...,...
1000,Winter in Tokyo berpusat pada kehidupan Ishida...,Romantis
1001,Markonah melarikan diri ke Jakarta karena akan...,Romantis
1002,"Tempat aking lebih dari 36 jam, Last Night ada...",Romantis
1003,Proyek baru ini adalah tentang seorang lelaki ...,Romantis


Karena label kita berupa data kategorikal, maka kita perlu melakukan proses one-hot-encoding. Jalankan kode di bawah untuk melakukan one-hot-encoding dan membuat dataframe baru.

In [3]:
category = pd.get_dummies(df.genre)
df_baru = pd.concat([df, category], axis=1)
df_baru = df_baru.drop(columns='genre')
df_baru

,ringkasan_sinopsis,Drama,Horor,Komedi,Laga,Romantis
0,Raden Mas Said putra sulung Tumenggung Wilarik...,1,0,0,0,0
1,Soe Hok Gie adalah seorang aktivis yang hidup ...,1,0,0,0,0
2,Guru Bangsa Tjokroaminoto menceritakan tentang...,1,0,0,0,0
3,POL menceritakan kisah hidup yang luar biasa d...,1,0,0,0,0
4,Perjalanan pahlawan Indonesia KH Ahmad Dahlan ...,1,0,0,0,0
...,...,...,...,...,...,...
1000,Winter in Tokyo berpusat pada kehidupan Ishida...,0,0,0,0,1
1001,Markonah melarikan diri ke Jakarta karena akan...,0,0,0,0,1
1002,"Tempat aking lebih dari 36 jam, Last Night ada...",0,0,0,0,1
1003,Proyek baru ini adalah tentang seorang lelaki ...,0,0,0,0,1


Agar dapat diproses oleh model, kita perlu mengubah nilai-nilai dari dataframe ke dalam tipe data numpy array menggunakan atribut values.

In [4]:
sinopsis = df_baru['ringkasan_sinopsis'].values
label = df_baru[['Drama', 'Horor', 'Komedi', 'Laga', 'Romantis']].values

Lalu, bagi data untuk training dan data untuk testing.

In [5]:
from sklearn.model_selection import train_test_split
sinopsis_latih, sinopsis_test, label_latih, label_test = train_test_split(sinopsis, label, test_size=0.2)

Kemudian kita ubah setiap kata pada dataset kita ke dalam bilangan numerik dengan fungsi Tokenizer. Setelah tokenisasi selesai, kita perlu membuat mengonversi setiap sampel menjadi sequence.

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
 
tokenizer = Tokenizer(num_words=5000, oov_token='x')
tokenizer.fit_on_texts(sinopsis_latih) 
tokenizer.fit_on_texts(sinopsis_test)
 
sekuens_latih = tokenizer.texts_to_sequences(sinopsis_latih)
sekuens_test = tokenizer.texts_to_sequences(sinopsis_test)
 
padded_latih = pad_sequences(sekuens_latih) 
padded_test = pad_sequences(sekuens_test)

Untuk arsitektur model kita menggunakan layer Embedding dengan dimensi embedding sebesar 16, serta dimensi dari input sebesar nilai num_words pada objek tokenizer. Jangan lupa panggil fungsi compile dan tentukan optimizer serta loss function yang akan dipakai oleh model.

In [7]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

Terakhir kita dapat mulai melatih model kita dengan memanggil fungsi fit().

In [8]:
num_epochs = 30
history = model.fit(padded_latih, label_latih, epochs=num_epochs, 
                    validation_data=(padded_test, label_test), verbose=2)

Epoch 1/30
26/26 - 8s - loss: 1.6099 - accuracy: 0.2127 - val_loss: 1.6128 - val_accuracy: 0.1493 - 8s/epoch - 306ms/step
Epoch 2/30
26/26 - 1s - loss: 1.6058 - accuracy: 0.2289 - val_loss: 1.6143 - val_accuracy: 0.1493 - 1s/epoch - 49ms/step
Epoch 3/30
26/26 - 1s - loss: 1.5622 - accuracy: 0.2836 - val_loss: 1.5974 - val_accuracy: 0.2090 - 1s/epoch - 48ms/step
Epoch 4/30
26/26 - 1s - loss: 1.3772 - accuracy: 0.4938 - val_loss: 1.5698 - val_accuracy: 0.2886 - 1s/epoch - 48ms/step
Epoch 5/30
26/26 - 1s - loss: 0.8883 - accuracy: 0.6891 - val_loss: 1.6987 - val_accuracy: 0.3184 - 1s/epoch - 48ms/step
Epoch 6/30
26/26 - 1s - loss: 0.7435 - accuracy: 0.7724 - val_loss: 1.8378 - val_accuracy: 0.3333 - 1s/epoch - 48ms/step
Epoch 7/30
26/26 - 1s - loss: 0.3675 - accuracy: 0.8968 - val_loss: 2.2667 - val_accuracy: 0.3134 - 1s/epoch - 48ms/step
Epoch 8/30
26/26 - 1s - loss: 0.1883 - accuracy: 0.9602 - val_loss: 2.6395 - val_accuracy: 0.3582 - 1s/epoch - 48ms/step
Epoch 9/30
26/26 - 1s - loss: 0

Akurasi dari model kita menunjukkan terjadinya overfitting karena akurasi pada data testing sangat besar, sedangkan akurasi pada data validasi jauh lebih kecil. Hal ini masih sangat luar biasa karena kita hanya memiliki 1000 buah sampel data!